In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset,concatenate_datasets
import transformers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer

from src.utils.myutils import *
import yaml

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

CS_DATA_PATH = PATH + '/data/CS/processed/BABE/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

BATCH_SIZE = 64
transformers.utils.logging.set_verbosity_error()

### BABE train_test split (SKIP IF DONE)

In [2]:
babe = load_dataset("csv", data_files=PATH + '/data/CS/raw/BABE/SG2.csv')['train']

Using custom data configuration default-41acc90be2294f89
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-41acc90be2294f89/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [3]:
babe = babe.train_test_split(0.15,seed=42)

Loading cached split indices for dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-41acc90be2294f89/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-2e89d302da86ff73.arrow and /home/horyctom/.cache/huggingface/datasets/csv/default-41acc90be2294f89/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-7a7ea8491428011d.arrow


In [4]:
babe['train'].to_csv(PATH + '/data/CS/processed/BABE/train.csv',index=False)
babe['test'].to_csv(PATH + '/data/CS/processed/BABE/test.csv',index=False) #THIS IS FOR THE FINAL MODEL SELECTED,TUNED

125614

## Load data

In [5]:
data = load_dataset('csv',data_files = CS_DATA_PATH)['train']
data

Using custom data configuration default-685bd5fc2d27ce6a


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/horyctom/.cache/huggingface/datasets/csv/default-685bd5fc2d27ce6a/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


Dataset({
    features: ['sentence', 'label'],
    num_rows: 3122
})

In [24]:
with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)

## Training

In [9]:
def compute_metrics_eval(eval_preds):
    metric = load_metric("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [22]:
training_args = TrainingArguments(
    num_train_epochs=10,
    per_device_train_batch_size=BATCH_SIZE,  
    #per_device_eval_batch_size=BATCH_SIZE,
    logging_steps=100,
    disable_tqdm = False,
    warmup_steps=40,
    save_total_limit=2,
    #evaluation_strategy="steps",
    #metric_for_best_model = 'f1',
    weight_decay=0.1,
    output_dir = './',
    learning_rate=5e-5)

## BEST PARAMS

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


### Cross-Val all models

In [23]:
model_scores = {}

for model_name in config_data['models']:
    scores = []
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    token_full = preprocess_data(data,tokenizer,'sentence')

    print("Running 5-fold CV on model: ",model_name,"...")
    for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

        token_train = Dataset.from_dict(token_full[train_index])
        token_valid = Dataset.from_dict(token_full[val_index])


        model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
        model.to(device)
        trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)#,,eval_dataset=token_valid,compute_metrics=compute_metrics_eval
        trainer.train()

        #evaluation
        eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
        scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        
    print("Done.")
    model_scores[model_name] = scores


loading configuration file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/1109a10013d6f221417ff52a1198bdc115db12e68d957a317c3cc463b17f5d43.28e3a761a3c5e32166573d4cc31b0a71ae1b56a98381799f8ff5332f26600fd7
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/vocab.txt f

Running 5-fold CV on model:  UWB-AIR/Czert-B-base-cased ...


loading configuration file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/1109a10013d6f221417ff52a1198bdc115db12e68d957a317c3cc463b17f5d43.28e3a761a3c5e32166573d4cc31b0a71ae1b56a98381799f8ff5332f26600fd7
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/pytorch_model.bin from cache at /home/horyctom/.cache/huggingface/transformers/6e

Step,Training Loss
100,0.517500
200,0.190600
300,0.048600
400,0.004200




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/UWB-AIR/Czert-B-base-cased/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/1109a10013d6f221417ff52a1198bdc115db12e68d957a317c3cc463b17f5d43.28e3a761a3c5e32166573d4cc31b0a71ae1b56a98381799f8ff5332f26600fd7
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "

...

  0%|          | 0/4 [00:00<?, ?ba/s]

Running 5-fold CV on model:  xlm-roberta-large ...


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/xlm-roberta-large/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/4d7a1550c9ab8701667b

Step,Training Loss


RuntimeError: CUDA out of memory. Tried to allocate 38.00 MiB (GPU 0; 31.75 GiB total capacity; 29.75 GiB already allocated; 2.00 MiB free; 30.90 GiB reserved in total by PyTorch)

In [26]:
model_scores

{'UWB-AIR/Czert-B-base-cased': [0.744,
  0.76,
  0.7275641025641025,
  0.7323717948717948,
  0.7403846153846154],
 'ufal/robeczech-base': [0.7840000000000001,
  0.776,
  0.7708333333333333,
  0.782051282051282,
  0.7628205128205128],
 'bert-base-multilingual-cased': [0.752,
  0.7408,
  0.7259615384615384,
  0.75,
  0.7259615384615384],
 'fav-kky/FERNET-C5': [0.7808,
  0.7648,
  0.7548076923076922,
  0.7564102564102565,
  0.7532051282051282],
 'fav-kky/FERNET-News': [0.7696,
  0.5104,
  0.7548076923076922,
  0.7788461538461539,
  0.7772435897435898],
 'DeepPavlov/bert-base-bg-cs-pl-ru-cased': [0.7632000000000001,
  0.7664,
  0.7291666666666665,
  0.7660256410256411,
  0.7772435897435898]}

In [31]:
for model in config_data['models']:
    print(model,"F1 score:",np.mean(model_scores[model]))

UWB-AIR/Czert-B-base-cased F1 score: 0.7408641025641025
ufal/robeczech-base F1 score: 0.7751410256410256
bert-base-multilingual-cased F1 score: 0.7389446153846153
fav-kky/FERNET-C5 F1 score: 0.7620046153846154
fav-kky/FERNET-News F1 score: 0.7181794871794871
DeepPavlov/bert-base-bg-cs-pl-ru-cased F1 score: 0.7604071794871794
